In [8]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cycler
import importlib

try:
    importlib.reload(dda)
except NameError:
    import dd_analysis as dda

In [ ]:
batchsize_runs = {
    'B5': 'data/batchsize/batch5_dd_res_2020-12-29_16-37-09.npz',
    'B10': 'data/batchsize/batch10_dd_res_2020-12-29_17-28-34.npz',
    'B15': 'data/batchsize/batch15_dd_res_2020-12-29_18-10-11.npz',
    'B16': 'data/batchsize/batch16_dd_res_2020-12-23_16-48-15.npz',
    'B20': 'data/batchsize/batch20_dd_res_2020-12-29_18-30-34.npz',
    'B25': 'data/batchsize/batch25_dd_res_2020-12-29_18-44-56.npz',
    'B30': 'data/batchsize/batch30_dd_res_2020-12-29_18-59-20.npz',
    'B31': 'data/batchsize/batch31_dd_res_2020-12-29_15-04-17.npz',
    'B32': 'data/batchsize/batch32_dd_res_2020-12-23_16-13-07.npz',
    'B35': 'data/batchsize/batch35_dd_res_2020-12-29_19-10-51.npz',
    'B40': 'data/batchsize/batch40_dd_res_2020-12-29_19-22-28.npz',
    'B45': 'data/batchsize/batch45_dd_res_2020-12-29_19-31-13.npz',
    'B50': 'data/batchsize/batch50_dd_res_2020-12-29_19-40-00.npz',
    'B55': 'data/batchsize/batch55_dd_res_2020-12-29_19-49-22.npz',
    'B60': 'data/batchsize/batch60_dd_res_2020-12-29_20-07-49.npz',
    'B64': 'data/batchsize/batch64_dd_res_2020-12-23_15-59-45.npz',
    'B65': 'data/batchsize/batch65_dd_res_2020-12-29_20-17-04.npz'
}

In [ ]:
domain_holdout_runs = {
    'original -> original': 'data/domain_ho_eqdist_dd_res_2020-12-24_14-56-49.npz',
    'original -> equidistant': 'data/domain_ho_orig_dd_res_2020-12-25_09-29-54.npz',
    'original -> 3-3-2': 'data/domain_ho_332_dd_res_2020-12-27_07-40-26.npz',
    'original -> 5-1-2': 'data/domain_ho_512_from_orig_dd_res_2021-01-03_11-43-15.npz',
    
    'equidistant -> original': 'data/domain_ho_orig_from_eqdist_dd_res_2021-01-02_06-57-29.npz',
    'equidistant -> equidistant': 'data/domain_ho_eqdist_from_eqdist_dd_res_2021-01-01_21-47-45.npz',
    'equidistant -> 3-3-2': 'data/domain_ho_332_from_eqdist_dd_res_2021-01-02_16-15-20.npz',
    'equidistant -> 5-1-2': 'data/domain_ho_512_from_eqdist_dd_res_2021-01-04_02-04-41.npz',
    
    '3-3-2 -> original': 'data/domain_ho_orig_from_332_dd_res_2020-12-31_02-11-40.npz',
    '3-3-2 -> equidistant': 'data/domain_ho_eqdist_from_332_dd_res_2020-12-30_20-25-13.npz',
    '3-3-2 -> 3-3-2': 'data/domain_ho_332_from_332_dd_res_2020-12-31_07-28-31.npz',
    '3-3-2 -> 5-1-2': 'data/domain_ho_512_from_332_dd_res_2021-01-03_18-39-16.npz',
    
    '5-1-2 -> original': 'data/domain_ho_orig_from_512_dd_res_2021-01-04_06-53-28.npz',
    '5-1-2 -> equidistant': 'data/domain_ho_eqdist_from_512_dd_res_2021-01-04_15-05-25.npz',
    '5-1-2 -> 3-3-2': 'data/domain_ho_332_from_512_dd_res_2021-01-04_12-36-34.npz',
    '5-1-2 -> 5-1-2': 'data/domain_ho_512_from_512_dd_res_2021-01-03_04-48-33.npz'
}

In [46]:
intergroup_runs = {
    'intergroup near -> near': 'data/intergroup/near_to_near_dd_res_2021-01-10_10-55-52.npz',
    'intergroup near -> far': 'data/intergroup/near_to_far_dd_res_2021-01-11_01-40-33.npz',
    'intergroup far -> near': 'data/intergroup/far_to_near_dd_res_2021-01-11_11-13-03.npz',
    'intergroup far -> far': 'data/intergroup/far_to_far_dd_res_2021-01-12_13-54-22.npz'
}

In [13]:
# Load data from previous runs

try:
    curr_res_paths = res_paths
except NameError:
    curr_res_paths = {}
    res_data = {}
    
res_paths = {
#     'Original': 'data/dd_res_2020-12-03_01-31-32.npz',
#     'Shared repr.': 'data/merged_repr_dd_res_2020-12-04_05-15-35.npz',
#    'Small item repr': 'data/small_item_repr_dd_res_2020-11-24_18-48-00.npz',
#    'More compressed': 'data/all_ratios_0.5_dd_res_2020-11-24_21-17-07.npz',
#    'Half-size HL': 'data/half_hidden_longer_dd_res_2020-11-25_01-06-37.npz',
#     'Merged item repr.': 'data/no_item_repr_reallocate_dd_res_2020-12-04_06-30-32.npz',
#     'Merged context repr.': 'data/no_ctx_repr_reallocate_dd_res_2020-12-04_07-48-18.npz',
#     'All merged': 'data/no_repr_reallocate_dd_res_2020-12-03_03-24-49.npz',
#     'Short original': 'data/short_save_params_dd_res_2020-11-26_11-44-16.npz',
#     'Short no item repr': 'data/short_save_params_no_item_repr_dd_res_2020-11-26_11-58-59.npz',
#     'Short no context repr': 'data/short_save_params_no_ctx_repr_dd_res_2020-11-26_12-05-47.npz'
#     'One domain': 'data/1domain_dd_res_2020-12-14_12-57-53.npz',
#     'Original (small init)': 'data/4domain_small_init_dd_res_2020-12-23_00-34-01.npz',
#     'One domain (small init)': 'data/1domain_small_init_dd_res_2020-12-23_00-52-41.npz',
#     **batchsize_runs,
#     **domain_holdout_runs,
#     'Original no reuse': 'data/original_no_reuse_dd_res_2021-01-06_17-15-11.npz',
#     '4-4 equal': 'data/4-4_dd_res_2021-01-05_14-16-02.npz',
#     '4-4 no reuse': 'data/4-4_no_reuse_dd_res_2021-01-06_14-51-28.npz',
#     '4-4 unequal': 'data/4-4_unequal_dd_res_2021-01-06_19-37-51.npz',
#     '4-4 equal independent': 'data/44_dho_dd_res_2021-01-09_06-31-41.npz',
#     'Original independent': 'data/422_dho_dd_res_2021-01-09_21-57-46.npz',
#     **intergroup_runs
    'Original new': 'data/original_new_dd_res_2021-01-12_20-21-39.npz',
    'Fixed biases': 'data/fixed_biases_dd_res_2021-01-12_20-30-39.npz',
    'All merged fixed biases': 'data/all_merged_fixed_biases_dd_res_2021-01-13_01-04-31.npz'
}

# Avoid reloading data whenever possible
try:
    res_data = {name: (res_data[name] if name in curr_res_paths and curr_res_paths[name] == path
                else dda.get_result_means(path, subsample_snaps=1, dist_metric='spearman', calc_all_repr_dists=False))
                for name, path in res_paths.items()}
except:
    # Assume loading failed, revert
    res_paths = curr_res_paths
    raise

In [14]:
# Get ready to look at some particular runs
# curr_runs = ['Original independent', '4-4 equal independent']
# curr_runs = [name for name in res_paths if name[0] == 'B' and int(name[1:]) % 5 == 0] # batch size runs
#curr_runs = intergroup_runs.keys()
curr_runs = res_paths.keys()

# cluster_types = ['original', 'equidistant', '3-3-2', '5-1-2']
# curr_runs = [c1 + ' -> ' + c2 for c1 in cluster_types for c2 in cluster_types]

curr_sets = {name: res_data[name] for name in curr_runs if name in res_data.keys()}
n_curr = len(curr_runs)

# colors supposed to correspond to accuracy in original generalization test (deprecated)
#prop_cycle = cycler(color=['chocolate', 'yellowgreen', 'royalblue', 'fuchsia', 'orangered'])

# general cycle that supports up to 40 lines
col_cycle = plt.rcParams['axes.prop_cycle']
line_cycle = cycler(linestyle=['-', '--', ':', '-.'])
prop_cycle = line_cycle * col_cycle
# line_cols = col_cycle[:3].by_key()['color']
# marker_cycle = cycler(mfc=line_cols, mec=line_cols)
# prop_cycle = marker_cycle * col_cycle[:3]

plt.close('all')

In [5]:
# Plot loss, accuracy, and test accuracy
plot_types = ['loss', 'weighted_acc']#, 'etg_domain']
fig, axs = dda.make_plot_grid(len(plot_types), 1, ax_dims=(9, 3),
                              ravel=True, prop_cycle=prop_cycle)

for i, (label, res) in enumerate(curr_sets.items()):
    for ax, ptype in zip(axs, plot_types):
        dda.plot_report(ax, res, ptype, label=label)
        
#     etg_ax = axs[2 + i // n_cluster_types]
#     dda.plot_report(etg_ax, res, 'etg_domain', label=label)

for ax in axs:
    dda.outside_legend(ax)#, fontsize='x-small', ncol=2)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
# axs[2].set_xscale('log')
# axs[2].get_legend().remove()
# ylims = axs[2].get_ylim()
# snap_epochs = curr_sets['Original']['snap_epochs']
# for ind in [0, 1, 5, 10, 50, 100]:
#     epoch = snap_epochs[ind]
#     axs[2].plot([epoch, epoch], ylims, 'k--')

# plt.show()

In [15]:
# Item RSA matrices

# run = [1]
# curr_sets_1run = {
#     name: dda.get_result_means(res_paths[name], runs=run)
#     for name in curr_sets.keys()
# }

inds_to_plot = range(0, 30, 2)
fig, axss = dda.auto_subplots(len(inds_to_plot), n_curr, ax_dims=(3.5, 3.5))
fig.suptitle('Mean RSA for items (all layers)')

for ind, axs in zip(inds_to_plot, axss):
    for (label, res), ax in zip(curr_sets.items(), axs):
        dda.plot_rsa(ax, res, 'item_full', ind, title_addon=label)#, item_order='group-outer')
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontsize('x-small')

fig.tight_layout(rect=(0, 0, 1, 0.98))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
# Dendrograms for just original
fig, axs = dda.auto_subplots(1, len(inds_to_plot), ax_dims=(4.5, 3.5))
for ind, ax in zip(inds_to_plot, axs.ravel()):
    dda.plot_repr_dendrogram(ax, curr_sets['Original'], 'item', ind)
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# Corresponding dendrograms
fig, axss = dda.auto_subplots(len(inds_to_plot), n_curr, ax_dims=(4, 3))
fig.suptitle('Item representation similarity')

for ind, axs in zip(inds_to_plot, axss):
    for (label, res), ax in zip(curr_sets.items(), axs):
        dda.plot_repr_dendrogram(ax, res, 'item_hidden', ind, title_addon=label)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Look at RSA projections onto group, domain, and item models
# First plot the models
item_models = dda.make_ortho_item_rsa_models(**next(iter(curr_sets.values()))['net_params'])

fig, axs = dda.auto_subplots(1, (len(item_models)), ax_dims=(3.4, 3.4))
for ax, (mtype, model) in zip(axs.ravel(), item_models.items()):
    dda.plot_rsa_model(ax, model, input_type='item')
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontsize('x-small') 
    ax.set_title(mtype + ' model RDM')
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
# Context hidden layer projections
# First plot models
ctx_models = dda.make_ortho_context_rsa_models(**next(iter(curr_sets.values()))['net_params'])

fig, axs = dda.auto_subplots(1, len(ctx_models), ax_dims=(3.4, 3.4))
for ax, (mtype, model) in zip(axs.ravel(), ctx_models.items()):
    dda.plot_rsa_model(ax, model, input_type='context')
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontsize('x-small') 
    ax.set_title(mtype + ' model RDM')
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
def plot_snap_projections(snap_type):
    n_plots = len(item_models) if 'item' in snap_type else len(ctx_models)
    fig, axs = dda.make_plot_grid(n_plots, 1, ax_dims=(9, 3),
                                  ravel=True, prop_cycle=prop_cycle)

    for label, res in curr_sets.items():
        if snap_type in res['repr_dists']:
            dda.plot_rdm_projections(res, snap_type, axs, label=label, normalize=True)

    for ax in axs:
        dda.outside_legend(ax)

    fig.tight_layout()
    plt.show()

# Item hidden layer projections
plot_snap_projections('item_hidden')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
# Item representation layer projections
plot_snap_projections('item')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# All layers item projection
plot_snap_projections('item_full')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
# Context hidden layer projections
plot_snap_projections('context_hidden')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
# Context representation layer projections
plot_snap_projections('context')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
# All layers context projection
plot_snap_projections('context_full')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# special analysis of projections onto cross_domain_group_match model to look at batch size
batch_sizes = np.array([res['train_params']['batch_size'] for res in curr_sets.values()])
projections = np.stack([np.mean(dda.get_rdm_projections(res, snap_type='item_full')['cross_domain_group_match'], axis=0)
                        for res in curr_sets.values()])

fig, ax = plt.subplots(figsize=(8, 6))
ax.matshow(projections)
ax.set_title('Strength of cross-domain (checkerboard) pattern')
ax.set_yticks(range(projections.shape[0]))
ax.set_yticklabels(batch_sizes)
ax.set_ylabel('Batch size')
ax.set_xticks(range(0, projections.shape[1], 4))
ax.set_xticklabels(curr_sets['B5']['snap_epochs'][::4])
ax.set_xlabel('Training epoch')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [100]:
# Fit linear models
reg_data = dda.make_dict_for_regression(curr_sets.values())

In [94]:
# Fit model for context domain information interacting with item domain and attribute similarity (both potentially important)
model1_res = dda.fit_linear_model('test_weighted_acc ~ ctx_same_vs_different_domain*(item_attribute_similarity + item_same_vs_different_domain)', reg_data)
print(model1_res.summary())

                            OLS Regression Results                            
Dep. Variable:      test_weighted_acc   R-squared:                       0.622
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     5971.
Date:                Tue, 08 Dec 2020   Prob (F-statistic):               0.00
Time:                        17:46:18   Log-Likelihood:                 21863.
No. Observations:               18180   AIC:                        -4.371e+04
Df Residuals:                   18174   BIC:                        -4.367e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [101]:
model2_res = dda.fit_linear_model('test_weighted_acc ~ item_attribute_similarity + ctx_same_vs_different_domain', reg_data)
print(model2_res.summary())

                            OLS Regression Results                            
Dep. Variable:      test_weighted_acc   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     7451.
Date:                Tue, 08 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:36:17   Log-Likelihood:                 18472.
No. Observations:               18180   AIC:                        -3.694e+04
Df Residuals:                   18177   BIC:                        -3.691e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [ ]:
# TODO: Try plotting a combination of item attribute similarity and context domain measures? Not sure if it makes sense.

In [74]:
# Context RSA matrix
inds_to_plot = [0, 1, 4, 11, 12, 30, 50, -1]
fig, axss = dda.auto_subplots(len(inds_to_plot), n_curr, ax_dims=(4, 4))
fig.suptitle('Mean RSA for contexts')

#ctx_sets = {name: res for name, res in curr_sets.items() if name in ['Original', 'Merged repr', 'No item repr']}

for ind, axs in zip(inds_to_plot, axss):
    for (label, res), ax in zip(curr_sets.items(), axs):
        dda.plot_rsa(ax, res, 'context_hidden', ind, title_addon=label, item_order='domain-inner')

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
# Corresponding context dendrograms
fig, axss = dda.auto_subplots(len(inds_to_plot), n_curr, ax_dims=(4, 3))
fig.suptitle('Context representation similarity')

for ind, axs in zip(inds_to_plot, axss):
    for (label, res), ax in zip(curr_sets.items(), axs):
        dda.plot_repr_dendrogram(ax, res, 'context', ind, title_addon=label)

fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
# Item rep RSA over all snapshots (useful maybe for finding interesting change points)
fig, axs = dda.make_plot_grid(len(curr_sets), 2, ax_dims=(5, 5))

for ax, (label, res) in zip(axs, curr_sets.items()):
    n_snaps = len(res['snap_epochs'])
    snap_freq = res['train_params']['snap_freq']
    im = ax.imshow(res['repr_dists']['item']['all'],
                   extent=(-0.5, n_snaps * snap_freq-0.5, n_snaps * snap_freq-0.5, -0.5))
    ax.set_xticks(res['snap_epochs'][::4])
    ax.tick_params(axis='x', labelrotation=45)
    ax.set_yticks(res['snap_epochs'][::4])
    ax.set_xlabel('Epochs')
    ax.set_title(f'Item repr distances over training ({label})')
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
# MDS of item representations over time
#for label, res in curr_sets.items():
dda.plot_repr_trajectories(curr_sets['Original'], 'item', dims=3)#, title_label=label)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
for label, res in curr_sets.items():
    dda.plot_repr_trajectories(res, 'context', dims=2, title_label=label)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
import importlib
importlib.reload(dda)

fig, axs = dda.make_plot_grid(len(curr_runs), 2, ax_dims=(6, 15))
run_num = 0
snap_index = 5

for (name, res), ax in zip(short_sets.items(), axs.ravel()):
    dda.plot_hl_input_pattern_correlations(ax, res, run_num, snap_index, title_label=name)
    
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …